In [1]:
import sys
import copy
import torch
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
sys.path.append("/home/zanardi/Codes/ML/RONEK/ronek/")
from ronek2 import env
from ronek2 import const
from ronek2 import backend as bkd
from ronek2.systems import BoxIso

In [3]:
env_opts = {
  "backend": "numpy",
  "device": "cpu",
  "device_idx": 0,
  "nb_threads": 2,
  "epsilon": None,
  "floatx": "float64"
}
env.set(**env_opts)

In [4]:
path_to_dtb = "/home/zanardi/Codes/ML/RONEK/ronek/examples/CR_Ar/database"

In [5]:
model = BoxIso(
  species={k: path_to_dtb + "/species/" + k + ".json" for k in ("Ar", "Arp", "em")},
  kin_dtb=path_to_dtb + "/rates/kin_fit.p",
  use_coll_int_fit=True,
  fixed_em=True
)
model.set_output(max_mom=2, linear=True)

In [6]:
sol_ref = "/home/zanardi/Workspace/AirDatabase/ThermoChemModels/StS/argon/testcase/test_iso/output_box"

In [ ]:
columns = "t X_em X_Ar X_Arp Th Te rho p n E".split()
box = np.loadtxt(sol_ref + "/box.dat")
box = pd.DataFrame(box, columns=columns)
box

In [8]:
t = box["t"].values
rho, Th0, Te0 = [box[k][0] for k in ("rho", "Th", "Te")]

In [9]:
def read_pop(name):
  ni_gi  = np.loadtxt(sol_ref + f"/pop_{name}.dat", comments="&", skiprows=2)[:,1]
  levels = model.mix.species[name].lev
  nb_pts = len(ni_gi) / len(levels["g"])
  ni_gi  = np.stack(np.split(ni_gi, nb_pts, axis=0), axis=0)
  return ni_gi * levels["g"].numpy()

In [10]:
n = {k: read_pop(k) for k in ("Ar", "Arp")}
n["em"] = (box["X_em"] * box["n"]).values.reshape(-1,1)
n_true = np.hstack([n[k] for k in model.species_order]).T
rho_true = model.mix.get_rho(bkd.to_torch(n_true)).numpy()
T_true = box[["Th", "Te"]].values.T
rho = np.sum(rho_true[:,0])

In [ ]:
y0 = np.zeros(model.nb_eqs+2)
y0[:model.mix.nb_comp] = rho_true[:,0] / rho
y0[-2] = Th0
y0[-1] = Te0
y0

In [ ]:
t_pred = t #np.geomspace(1e-12, 1e3)
y, runtime = model.solve_fom(t_pred, y0, rho)
runtime

In [13]:
w_pred = copy.deepcopy(y[:model.mix.nb_comp])
n_pred = model.mix.get_n(bkd.to_torch(w_pred)).numpy()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  for i in s.indices:
    plt.loglog(t[1:], n_true[i][1:])
    plt.loglog(t_pred[1:], n_pred[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  plt.show()
  plt.close()

In [ ]:
yeq = w_pred[:,-1]
yeq_prim = model.equil.from_prim(rho, Th0, Te0)
yeq_prim, yeq

In [ ]:
ylin0 = y0[:-2] - yeq
A = model.jac(0.0, yeq)
b = model.fun(0.0, yeq)
b

In [17]:
ylin = [sp.linalg.expm(A*ti) @ ylin0 + yeq for ti in t]
ylin = np.vstack(ylin).T

In [18]:
w_lin = copy.deepcopy(ylin[:model.mix.nb_comp])
n_lin = model.mix.get_n(bkd.to_torch(w_lin)).numpy()

In [ ]:
for s in model.mix.species.values():
  print(s.name)
  err = []
  for i in s.indices:
    ierr = np.mean(np.abs((n_true[i] - n_lin[i])/n_pred[i]))
    err.append(ierr)
    plt.loglog(t[1:], n_pred[i][1:])
    plt.loglog(t[1:], n_lin[i][1:], ls="--", color=plt.gca().lines[-1].get_color())
  print(np.mean(err))
  plt.show()
  plt.close()